<a href="https://colab.research.google.com/github/koushik1234/machine_learning/blob/master/hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import boston_housing
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU
from sklearn.preprocessing import StandardScaler
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import warnings

Using TensorFlow backend.


In [3]:
#register swish as a custom object in keras
warnings.filterwarnings(action='ignore')
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

get_custom_objects().update({'swish': Activation(swish)})

In [14]:
#load the data
from google.colab import files
uploaded=files.upload()

Saving Churn_Modelling.csv to Churn_Modelling (3).csv


In [0]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [7]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
dataset.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [16]:
X.shape

(10000, 10)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
#onehotencoder = OneHotEncoder()
#X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]


In [18]:
X.shape

(10000, 9)

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [20]:
X_train.shape,X_test.shape

((8000, 9), (2000, 9))

In [24]:
'''
def create_model(layers=20, activation='sigmoid'):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)
'''

"\ndef create_model(layers=20, activation='sigmoid'):\n    model = Sequential()\n    for i, nodes in enumerate(layers):\n        if i==0:\n            model.add(Dense(nodes,input_dim=X_train.shape[1]))\n            model.add(Activation(activation))\n        else:\n            model.add(Dense(nodes))\n            model.add(Activation(activation))\n    model.add(Dense(1)) # Note: no activation beyond this point\n    \n    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])\n    return model\n    \nmodel = KerasClassifier(build_fn=create_model, verbose=0)\n"

In [0]:
# let's create a function that creates the model (required for KerasClassifier) 
# while accepting the hyperparameters we want to tune 
# we also pass some default values such as optimizer='rmsprop'
def create_model(init_mode='uniform'):
    # define model
    model = Sequential()
    model.add(Dense(64, kernel_initializer=init_mode, activation='relu', input_dim=X_train.shape[1])) 
    model.add(Dropout(0.1))
    model.add(Dense(64, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [26]:
%%time
seed = 7
np.random.seed(seed)
batch_size = 128
epochs = 10

model_CV = KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=1)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 
             'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





8000/8000 [==============================] - 1s 182us/step - loss: 0.4919 - acc: 0.7836
Epoch 2/10
8000/8000 [==============================] - 0s 31us/step - loss: 0.4125 - acc: 0.8257
Epoch 3/10
8000/8000 [==============================] - 0s 30us/step - loss: 0.3934 - acc: 0.8319
Epoch 4/10
8000/8000 [==============================] - 0s 29us/step - loss: 0.3820 - acc: 0.8401
Epoch 5/10
8000/8000 [==============================] - 0s 28us/step - loss: 0.3748 - acc: 0.8448
Epoch 6/10
8000/8000 [==============================] - 0s 29us/step - loss: 0.3682 - acc: 0.8479
Epoch 7/10
8000/8000 [==============================] - 0s 28us/step - loss: 0.3627 - acc: 0.8492
Epoch 8/10
8000/8000 [==============================] - 0s 30us/step - loss: 0.3565 - acc: 

In [27]:
# print results
print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for 0.8563735890481236 using {'init_mode': 'he_uniform'}
 mean=0.8311, std=0.006308 using {'init_mode': 'uniform'}
 mean=0.8497, std=0.002967 using {'init_mode': 'lecun_uniform'}
 mean=0.8414, std=0.009328 using {'init_mode': 'normal'}
 mean=0.796, std=0.003717 using {'init_mode': 'zero'}
 mean=0.853, std=0.005253 using {'init_mode': 'glorot_normal'}
 mean=0.8527, std=0.002283 using {'init_mode': 'glorot_uniform'}
 mean=0.8462, std=0.00507 using {'init_mode': 'he_normal'}
 mean=0.8564, std=0.008005 using {'init_mode': 'he_uniform'}


In [0]:
'''
layers = [[20], [40, 20], [45, 30, 15]]
#layers=[20]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)
'''